### PaddleOCR with Preprocessing

In [8]:
import os
import json
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import time
from paddleocr import PaddleOCR
import numpy as np
import Levenshtein as lev
from tqdm import tqdm



annotations_folder = '/home/maxkhamuliak/projects/OCR-comparsion/annotations'
images_folder = '/home/maxkhamuliak/projects/OCR-comparsion/images'

data = []

for filename in os.listdir(annotations_folder):
    if filename.endswith('.json'):
        with open(os.path.join(annotations_folder, filename), 'r') as file:
            annotation_data = json.load(file)
            image_filename = filename.split('.')[0] + '.png'
            text = ""
            for item in annotation_data['form']:
                text += item['text'] + " "

            data.append({'filename': image_filename, 'text': text.strip()})

df = pd.DataFrame(data)

df.head()


,filename,text
0,82092117.png,TO: DATE: 3 Fax: NOTE: 82092117 614 -466 -5087...
1,82200067_0069.png,TO: FROM: x SUBJECT: DIVISION: DIVISION: DIV...
2,82250337_0338.png,TO: FROM: DATE: MANUFACTURER: BRAND: Oct. Dec....
3,82251504.png,17 cc: : From: Area: Region: 5 X Chains: Indep...
4,82252956_2958.png,AUG 4 SEP 15 JUN 23 MAY 12 REGION: DIVISION: 7...


#### Preprocessing 

In [9]:
def resize_image(image, target_size):
    return cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)

def binarize_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary

def reduce_noise(image):
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    return cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)

def correct_skew(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    coords = np.column_stack(np.where(gray > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

def apply_morphology(image):
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel, iterations=1)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=1)
    return closing

def sharpen_image(image):
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharpened = cv2.filter2D(image, -1, kernel)
    return sharpened

def preprocess_image(image, techniques):
    preprocessed = image.copy()
    for technique in techniques:
        if technique == 'resize':
            preprocessed = resize_image(preprocessed, (800, 600))
        elif technique == 'binarization':
            preprocessed = binarize_image(preprocessed)
        elif technique == 'noise_reduction':
            preprocessed = reduce_noise(preprocessed)
        elif technique == 'skew_correction':
            preprocessed = correct_skew(preprocessed)
        elif technique == 'morphology':
            preprocessed = apply_morphology(preprocessed)
        elif technique == 'sharpening':
            preprocessed = sharpen_image(preprocessed)
    return preprocessed

In [10]:
def evaluate_ocr(image_path, text, preprocessing_techniques):

    image = cv2.imread(image_path)
    preprocessed = preprocess_image(image, preprocessing_techniques)
    
    ocr_model = PaddleOCR(lang='en', use_gpu=False)
    
    start_time = time.time()
    result = ocr_model.ocr(preprocessed, cls=True)
    end_time = time.time()
    execution_time = end_time - start_time
    
    predicted_text = ' '.join([line[1][0] for line in result[0]])
    
    accuracy = len(set(predicted_text.split()) & set(text.split())) / len(set(text.split()))
    iou = len(set(predicted_text.split()) & set(text.split())) / len(set(predicted_text.split()) | set(text.split()))
    levenshtein_distance = lev.distance(predicted_text, text)  
    
    
    return accuracy, iou, levenshtein_distance, execution_time

In [11]:
preprocessing_techniques = [
    [],
    ['resize'],
    ['binarization'],
    ['noise_reduction'],
    ['skew_correction'],
    ['morphology'],
    ['sharpening'],
    ['resize', 'binarization'],
    ['resize', 'noise_reduction'],
    ['resize', 'skew_correction'],
    ['resize', 'morphology'],
    ['resize', 'sharpening']
]

In [12]:
results = []
for techniques in tqdm(preprocessing_techniques):
    accuracy_scores = []
    iou_scores = []
    levenshtein_scores = []
    execution_times = []
    
    for _, row in df.iterrows():
        image_path = os.path.join(images_folder, row['filename'])
        accuracy, iou, levenshtein, execution_time = evaluate_ocr(image_path, row['text'], techniques)
        accuracy_scores.append(accuracy)
        iou_scores.append(iou)
        levenshtein_scores.append(levenshtein)
        execution_times.append(execution_time)
    
    results.append({
        'preprocessing': ' + '.join(techniques),
        'accuracy': np.mean(accuracy_scores),
        'iou': np.mean(iou_scores),
        'levenshtein': np.mean(levenshtein_scores),
        'execution_time': np.mean(execution_times)
    })

  0%|          | 0/12 [00:00<?, ?it/s]

[2024/06/14 19:26:56] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/home/maxkhamuliak/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/maxkhamuliak/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thres

  8%|▊         | 1/12 [05:37<1:01:48, 337.11s/it]

[2024/06/14 19:32:34] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/home/maxkhamuliak/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/maxkhamuliak/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thres

 17%|█▋        | 2/12 [11:07<55:34, 333.41s/it]  

...

 83%|████████▎ | 10/12 [55:31<10:06, 303.34s/it]

[2024/06/14 20:22:28] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/home/maxkhamuliak/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/maxkhamuliak/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thres

 92%|█████████▏| 11/12 [58:03<04:17, 257.24s/it]

[2024/06/14 20:25:00] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/home/maxkhamuliak/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/maxkhamuliak/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thres

100%|██████████| 12/12 [1:03:16<00:00, 316.39s/it]


#### Evaluate OCR performance without preprocessing


In [13]:
accuracy_scores = []
iou_scores = []
levenshtein_scores = []
execution_times = []

for _, row in df.iterrows():
    image_path = os.path.join(images_folder, row['filename'])
    accuracy, iou, levenshtein, execution_time = evaluate_ocr(image_path, row['text'], [])
    accuracy_scores.append(accuracy)
    iou_scores.append(iou)
    levenshtein_scores.append(levenshtein)
    execution_times.append(execution_time)


results.append({
    'preprocessing': 'No Preprocessing',
    'accuracy': np.mean(accuracy_scores),
    'iou': np.mean(iou_scores),
    'levenshtein': np.mean(levenshtein_scores),
    'execution_time': np.mean(execution_times)
})


[2024/06/14 20:30:13] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/home/maxkhamuliak/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/maxkhamuliak/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thres

### Comparsion df

In [14]:
results_df = pd.DataFrame(results)
results_df['accuracy_rank'] = results_df['accuracy'].rank(ascending=False)
results_df['iou_rank'] = results_df['iou'].rank(ascending=False)
results_df['levenshtein_rank'] = results_df['levenshtein'].rank(ascending=True)
results_df['execution_time_rank'] = results_df['execution_time'].rank(ascending=True)
results_df['overall_rank'] = (results_df['accuracy_rank'] + results_df['iou_rank'] + results_df['levenshtein_rank'] + results_df['execution_time_rank']) / 4
results_df = results_df.sort_values('overall_rank')

# Display the comparison DataFrame
print("Comparison of Preprocessing Techniques:")
print(results_df)

# Display the top performing techniques
top_techniques = results_df.head(5)
print("\nTop Performing Preprocessing Techniques:")
print(top_techniques)

Comparison of Preprocessing Techniques:
               preprocessing  accuracy       iou  levenshtein  execution_time  \
3            noise_reduction  0.643076  0.518970       437.08        4.423373   
0                             0.635716  0.513982       437.42        4.289124   
12          No Preprocessing  0.635716  0.513982       437.42        7.974113   
6                 sharpening  0.483891  0.351973       489.72        4.261531   
4            skew_correction  0.571919  0.485145       789.18        3.633826   
8   resize + noise_reduction  0.442865  0.346106       605.06        3.744298   
2               binarization  0.461286  0.324280       495.24        5.992815   
1                     resize  0.409821  0.320621       620.88        4.319919   
9   resize + skew_correction  0.331023  0.264029       841.72        2.645292   
11       resize + sharpening  0.205711  0.136035       664.22        4.115162   
5                 morphology  0.008037  0.007152      1014.36        